In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# English

In [2]:
data=pd.read_pickle('C:/Users/forgo/Desktop/dsa4266/full_df_en_processed.pkl').reset_index()#.drop_duplicates('clean_msg')
temp=pd.read_pickle('C:/Users/forgo/Desktop/dsa4266/full_df.pkl').reset_index()
data=data.merge(temp[['doc_id','pretranslation']],how='left',on='doc_id').drop_duplicates('pretranslation').drop_duplicates('clean_msg')
# data=data[['class','processed']].drop_duplicates().rename(columns={'processed':'clean_msg'})
data['lang']=data['doc_id'].apply(lambda x: x[6])
data=data[data['clean_msg']!=''].reset_index()
data

,index,doc_id,class,translated,clean_msg,pretranslation,lang
0,1,trec06c/data/000/007,spam,Dear person in charge (manager/finance): Hello...,dear person charge hello trading co ltd strong...,尊敬的负责人（经理／财务）：您好！ 我是深圳伟仕嘉贸易有公司：兴办贸易、物资供销，实力雄厚；...,c
1,2,trec06c/data/000/014,spam,Hello! Nice to meet you. Our company is intere...,hello nice meet company interested cooperating...,"您好！很高兴认识您。我司有意与您们合作:可长久给您们带来 巨大的效益,另因合作项目的高度自动...",c
2,3,trec06c/data/000/005,spam,TO: Your company’s manager and finance manager...,manager finance manager hello trading co ltd b...,TO：贵公司经理、财务 您好！ 深圳市春洋贸易有限公司（东莞分公司）...,c
3,4,trec06c/data/000/013,spam,New remote-controlled airplanes are widely sol...,new airplanes widely sold many types toy airpl...,新型遥控飞机销路广 百元可办厂 玩具飞机品种繁多，但用微型小开关控制的新型能在空中自控飞翔的...,c
4,5,trec06c/data/000/021,spam,"Dear company (factory) manager, hello: Our com...",dear company factory manager hello company com...,尊敬的公司（工厂）经理负责人你好： 我公司是一家多年为外资企业代理进出口业务的公司，现有部分...,c
...,...,...,...,...,...,...,...
39240,42142,trec06p/data/035/115,spam,Rent WITHOUT COMMISSION\r \r m. Avtozavodskaya...,rent without commission min foot blocks buildi...,"Аренда БЕЗ КОМИССИОННЫХ\r \r м. Автозаводская,...",p
39241,42144,trec06p/data/058/203,ham,RIFFx~ WAVEfmt + + dataS~ ~y{yw{ zuvx}}ytmjhed...,aa aa tr pf en kn la qu oka pk oz gun nom mum ...,RIFFx~ WAVEfmt      +  +    dataS~ ...,p
39242,42145,trec06p/data/014/167,spam,dangerous cork blob thirteenth alliterate argu...,dangerous cork blob thirteenth alliterate argu...,dangerous cork blob thirteenth alliterate argu...,p
39243,42146,trec06p/data/025/231,spam,:*.☆. o:☆';*. :*.☆. o:☆';*. :*.☆. o:☆';*. :*.☆...,largest online community history born site abs...,\r :*.☆。o:☆';*。:*.☆。o:☆';*。:*.☆。o:☆';*。:*.☆...,p


In [3]:
# en_regex = re.compile('[^a-zA-Z ]')
# d = enchant.Dict("en_US") 
# STOPWORDS = stopwords.words('english')

In [4]:
# def enPreprocess(string):
#     string=string.replace('\n',' ') #remove newline char
#     string=en_regex.sub('',string) #removes non alphabets
#     string=string.split()
#     string=[word.lower() for word in string if len(word)>1]
#     string=' '.join([word for word in string if d.check(word) and (word not in STOPWORDS)]) # split the string into list and check each word if it is in the english dictionary and longer than 1 alphabet
#     return string

In [5]:
# data['clean_msg']=data['translated'].apply(lambda x: enPreprocess(x))
# data.to_pickle('lowercase words.pkl')

In [6]:

X = data.drop(columns='class')
y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.2)

In [7]:
X_train_e=X_train[X_train['lang']=='p']['clean_msg']
X_test_e=X_test[X_test['lang']=='p']['clean_msg']
y_train_e=y_train.loc[X_train_e.index]
y_test_e=y_test.loc[X_test_e.index]

X_train_c=X_train[X_train['lang']=='c']['pretranslation']
X_test_c=X_test[X_test['lang']=='c']['pretranslation']
y_train_c=y_train.loc[X_train_c.index]
y_test_c=y_test.loc[X_test_c.index]

X_train=X_train['clean_msg']
X_test=X_test['clean_msg']

In [8]:
vect = CountVectorizer(stop_words='english')
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm= vect.transform(X_test)


In [9]:
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

print('translated dataset Multinomial Naive Bayes ')
print('acc',metrics.accuracy_score(y_test, y_pred_class))
print('f1',f1_score(y_test.to_list(), y_pred_class,pos_label="spam"))
# metrics.confusion_matrix(y_test, y_pred_class)

translated dataset Multinomial Naive Bayes 
acc 0.9308192126385527
f1 0.83757104397248


In [10]:
logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train_dtm, y_train)
y_pred_class = logreg.predict(X_test_dtm)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]

print('translated dataset log regression ')
print('acc',metrics.accuracy_score(y_test, y_pred_class))
print('f1',f1_score(y_test.to_list(), y_pred_class,pos_label="spam"))

translated dataset log regression 
acc 0.9709517135940884
f1 0.9341421143847488


# 1 model for each language

In [11]:
vect = CountVectorizer(stop_words='english')
X_train_e_dtm = vect.fit_transform(X_train_e)
X_test_e_dtm= vect.transform(X_test_e)

In [12]:
import re
cn_word=re.compile("[\u4e00-\u9FFF]")
X_train_c=X_train_c.apply(lambda x: ''.join([word for word in x if cn_word.match(word)]))
X_test_c=X_test_c.apply(lambda x: ''.join([word for word in x if cn_word.match(word)]))

In [13]:
vect = CountVectorizer(tokenizer=lambda txt:[*txt])
X_train_c_dtm = vect.fit_transform(X_train_c)
X_test_c_dtm= vect.transform(X_test_c)

C:\Users\forgo\AppData\Roaming\Python\Python310\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [14]:

nb = MultinomialNB()
nb.fit(X_train_e_dtm, y_train_e)
y_pred_e_class = nb.predict(X_test_e_dtm)


nb = MultinomialNB()
nb.fit(X_train_c_dtm, y_train_c)
y_pred_c_class = nb.predict(X_test_c_dtm)

In [15]:
print('2 model combined Multinomial Naive Bayes ')
y_pred_combined=y_pred_e_class.tolist()+y_pred_c_class.tolist()
y_actual_combined=pd.concat([y_test_e,y_test_c]).to_list()
print('acc',metrics.accuracy_score(y_actual_combined, y_pred_combined))
print('f1',f1_score(y_pred_combined,y_actual_combined,pos_label="spam"))

2 model combined Multinomial Naive Bayes 
acc 0.9375716651802778
f1 0.8642659279778393


In [16]:

logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train_e_dtm, y_train_e)
y_pred_e_class = logreg.predict(X_test_e_dtm)
# y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]

logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train_c_dtm, y_train_c)
y_pred_c_class = logreg.predict(X_test_c_dtm)
# y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]

In [17]:

print('2 model combined log regression ')
y_pred_combined=y_pred_e_class.tolist()+y_pred_c_class.tolist()
y_actual_combined=pd.concat([y_test_e,y_test_c]).to_list()
print('acc',metrics.accuracy_score(y_actual_combined, y_pred_combined))
print('f1',f1_score(y_pred_combined,y_actual_combined,pos_label="spam"))

2 model combined log regression 
acc 0.9782137851955663
f1 0.9505065123010129
